In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

from ast import literal_eval

from urllib.parse import urlparse

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances
from sklearn.metrics import jaccard_score
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.model_selection import train_test_split, RandomizedSearchCV

import string
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer as PS

import joblib

import pprint

import pickle

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
# data = pd.read_csv('dataset/full_dataset.csv')
# data = pd.read_csv('../dataset/full_dataset.csv')

In [3]:
# toy_data = data.sample(frac=0.1, replace=False)
# toy_data.head(10)

In [4]:
# toy_data.info()

In [5]:
# def literal_return(val):
#     try:
#         return literal_eval(val)
#     except (ValueError, SyntaxError) as e:
#         return val

# def df_str_to_literal(df, column_name):
#     df[column_name] = df[column_name].apply(literal_return)
#     return df[column_name]

In [6]:
# def clean_df(df, columns_list):
#     for col in columns_list:
#         df[col] = df_str_to_literal(df, col)
#     return None

In [7]:
# clean_df(toy_data, ['ingredients', 'directions', 'NER'])

In [8]:
# toy_data.head()

In [9]:
# toy_data['bag_of_words'] = ''
# columns_list = ['title', 'ingredients', 'directions']
# for col in columns_list:
#     if col == 'title':
#         toy_data['bag_of_words'] += toy_data[col] + ' '
#     if col == 'ingredients':
#         toy_data['bag_of_words'] += toy_data[col].apply(' '.join) + ' '
#     if col == 'directions':
#         toy_data['bag_of_words'] += toy_data[col].apply(' '.join)
# toy_data.head()

In [10]:
# bow_toy_data = toy_data[['title','bag_of_words']]
# bow_toy_data.head()

In [11]:
# documents = bow_toy_data.bag_of_words

In [12]:
# def clean_document(document):
#     '''
#     Takes in a string.
#     Returns cleaned string.
#     '''
#     # lowercase the strings
#     doc_lower = document.lower() 

#     #tokenize
#     tokens = word_tokenize(doc_lower) 
    
#     # remove punctuation
#     punc = set(string.punctuation)
#     tokens_no_punc = [word for word in tokens if word not in punc]
   
#     # remove stopwords
#     s_words = set(stopwords.words('english'))
#     s_words_list = ['tablespoon', 'tbsp', 'teaspoon', 'tsp', 'cup', 'oz', 'lb', 'c.']
#     for word in s_words_list:
#         s_words.add(word)
#     tokens_no_sw = [word for word in tokens_no_punc if word not in s_words]
    
#     # stem the words to get rid of multiple forms of the same word
#     porter = PS()
#     tokens_stemmed = [porter.stem(word) for word in tokens_no_sw]
    
#     # join all words into one string
#     cleaned_doc = ' '.join(tokens_stemmed)
    
#     return cleaned_doc

In [13]:
# bow_toy_data['cleaned_bow'] = toy_data['bag_of_words'].apply(clean_document)
# bow_toy_data

In [14]:
# docs_cleaned = bow_toy_data['cleaned_bow']

In [15]:
lda = joblib.load('./models/lda_model_6_tid_pickle4.joblib')
tf_vectorizer = joblib.load('./models/vec_6_tid_pickle4.joblib')

In [16]:
with open('./flask_app/df.pkl', 'rb') as f:
    df = pickle.load(f)

In [17]:
docs = df['cleaned_bow']

In [18]:
tf = tf_vectorizer.fit_transform(docs)

In [19]:
# lda.perplexity(tf)

In [20]:
probs = lda.transform(tf)

In [21]:
def get_keyword_idxs(keyword, idx_arr, recipes):
    keyword_recipes = recipes[recipes.str.contains(keyword, case=False, regex=False)]
    keyword_samples = np.random.choice(keyword_recipes.index, size=min(len(keyword_recipes), 50), replace=False)
    keyword_idxs = []
    for sample_idx in keyword_samples:
        keyword_idx = int(np.where(idx_arr == sample_idx)[0])
        keyword_idxs.append(keyword_idx)
    return keyword_idxs

In [22]:
# def get_sample_recipes(keyword, recipes):
#     keyword_idxs = get_keyword_idxs(keyword, recipes)
#     print(f'Recipe samples for {keyword}')
#     return np.array(recipes)[keyword_idxs]

In [23]:
def closest_recipes(keyword, idx_arr, recipes, probs, n_recipes=10):
    keyword_idxs = get_keyword_idxs(keyword, idx_arr, recipes)
    
    d={}
    for idx in keyword_idxs:
        sims = cosine_distances(probs[idx].reshape(1, -1), probs).argsort()[0]
        for sim in sims[1:n_recipes+1]:
            if sim not in d:
                d[sim] = 1
            else:
                d[sim] += 1
                
    rec_idxs = [k for k, v in sorted(d.items(), key=lambda item: item[1])][:-n_recipes:-1]
    recipe_recs = np.array(recipes)[rec_idxs]
    reference_recipes = np.array(recipes)[keyword_idxs]
    
    return recipe_recs, rec_idxs, reference_recipes, keyword_idxs

In [24]:
def get_results_df(recipe_recs, links, rec_idxs):
    rec_links = np.array(links)[rec_idxs]
    results = {'Recipe': recipe_recs, 'Link': rec_links}
    results_df = pd.DataFrame(results)
    return results_df

In [36]:
def results(keyword=''):
    if keyword == '':
        return 'You must enter a keyword.'
    recipe_recs, rec_idxs, reference_recipes, keyword_idxs = closest_recipes(keyword, idx_arr, recipes, probs)
    if len(recipe_recs) < 1:
        return 'Keyword not found.'
    results_df = get_results_df(recipe_recs, links, rec_idxs)
    return results_df

In [37]:
recipes = df['title']

In [38]:
idx_arr = np.array(recipes.index)

In [39]:
links = df['link']

In [40]:
keyword = 'asbestos'

In [41]:
results_df = results(keyword)

In [42]:
results_df

'Keyword not found.'

In [ ]:
# keyword = 'pizza'
# n_recipes = 10
# recipe_recs, rec_idxs, reference_recipes, keyword_idxs = closest_recipes(keyword, recipes, probs, n_recipes=n_recipes)
# print(f'Top {n_recipes} recipes for "{keyword}":')
# print(recipe_recs)
# print('')
# print(f'Based on recipe samples:')
# print(reference_recipes)

In [ ]:
# bow_toy_data['link'] = toy_data['link']
# bow_toy_data.head()

In [ ]:
# pd.to_pickle(bow_toy_data, 'df.pkl')